In [1]:
import torch
import torchvision

In [2]:
dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=torchvision.transforms.ToTensor())


C:\Users\ashua\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
train_ds, val_ds = torch.utils.data.random_split(dataset, [50000, 10000])

print(len(train_ds), len(val_ds))

50000 10000


In [4]:
from torch.utils.data.dataloader import DataLoader

batch_size = 100

train_dl = DataLoader(train_ds, batch_size)
val_dl = DataLoader(val_ds, batch_size)

In [5]:
input_size = 28*28
num_classes = 10

class MnistModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(input_size, num_classes)
        
    def forward(self, image):
        image = image.reshape(-1,784)
        return self.linear(image)
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)
        loss = torch.nn.functional.cross_entropy(out, labels) 
        return loss

    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    
        loss = torch.nn.functional.cross_entropy(out, labels)   
        acc = accuracy(out, labels)           
        return {'loss': loss, 'accuracy': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accuracy = [x['accuracy'] for x in outputs]
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {'loss': epoch_loss.item(), 'accuracy': epoch_accuracy.item()}
        
    def epoch_end(self, epoch, result):
        print("Epoch [{}], loss: {:.4f}, accuracy: {:.4f}".format(epoch, result['loss'], result['accuracy']))
        
model = MnistModel()    

print(model.linear.weight.shape,model.linear.bias.shape)
list(model.parameters())

torch.Size([10, 784]) torch.Size([10])


[Parameter containing:
 tensor([[-0.0279, -0.0081,  0.0317,  ...,  0.0287, -0.0192, -0.0255],
         [-0.0147,  0.0100,  0.0053,  ...,  0.0168,  0.0075,  0.0105],
         [ 0.0068, -0.0349, -0.0278,  ...,  0.0063,  0.0206, -0.0315],
         ...,
         [-0.0283,  0.0029,  0.0240,  ..., -0.0198, -0.0153,  0.0285],
         [ 0.0147,  0.0110,  0.0332,  ...,  0.0307,  0.0042,  0.0001],
         [ 0.0353, -0.0085,  0.0120,  ..., -0.0028, -0.0171,  0.0236]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0234,  0.0344, -0.0304, -0.0197, -0.0353, -0.0178, -0.0319, -0.0106,
          0.0224, -0.0125], requires_grad=True)]

In [6]:
for images, label in train_dl:
    print(images.shape)
    outputs = model(images)
    break

torch.Size([100, 1, 28, 28])


In [7]:
def accuracy(outputs, label):
    preds = torch.max(outputs, dim=1)

    return (sum(preds[1] == label)/ len(label))

print(accuracy(outputs,label))

tensor(0.1200)


In [8]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = [] 
    
    for epoch in range(epochs):
        
        for batch in train_loader:
            loss = model.training_step(batch)
            
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
        
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [9]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [10]:
result0 = evaluate(model, val_dl)
print(result0)

In [ ]:
history1 = fit(3, 0.001, model, train_dl, val_dl)

Epoch [0], loss: 1.8727, accuracy: 0.6659
Epoch [1], loss: 1.5660, accuracy: 0.7601
Epoch [2], loss: 1.3530, accuracy: 0.7873


In [ ]:
from PIL import Image
test_dataset = torchvision.datasets.MNIST(root="./data", train=False)

x=4


img, label = test_dataset[x]
img.resize((280,280)).show()

test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=torchvision.transforms.ToTensor())

def predict_image(img, model):
    xb = img.unsqueeze(0)
    yb = model(xb)
    print(yb)
    _, preds = torch.max(yb, dim=1)
    print(preds)
    print(_)
    return preds[0].item()

img, label = test_dataset[x]
predict_image(img, model)


tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

4